## 13.3 カテゴリ変数を使うモデル
カテゴリ変数を用いた```statsmodels```と```sklearn```による重回帰分析のスニペット。

```polars```の```to_dummies()```関数は数値型もダミー変数にするので注意。

### (共通)データの読み込み
```seaborn```の```tips```データセットを用いる。

In [ ]:
import polars as pl
import seaborn as sns

# データの読み込み
tips = pl.DataFrame(sns.load_dataset("tips"))
display(tips.head())

# データ型を表示する
display(tips.columns)
display(tips.dtypes)

# カテゴリ変数のユニーク値を表示する
display(tips["sex"].unique().to_numpy())
display(tips["smoker"].unique().to_numpy())
display(tips["day"].unique().to_numpy())
display(tips["time"].unique().to_numpy())

### 13.2.1 statsmodels

In [ ]:
import statsmodels.formula.api as smf

# モデルを学習する
model = smf.ols(formula = "tip ~ total_bill + size + sex * smoker + day + time", data = tips).fit()

# 学習結果を表示する
display(model.summary())

# 係数を表示する
display(model.params)

# 信頼区間を表示する
display(model.conf_int())

### 13.1.2 sklearn

In [ ]:
from sklearn import linear_model

# ダミー変数を作成する
import polars.selectors as cs
# 数値型は外しておく
df_numeric = tips.select(cs.numeric()).drop("tip")
# カテゴリ変数のみダミー変数を作成する
df_categorical = tips.select(cs.categorical())
df_dummy = df_categorical.to_dummies(drop_first = True)
# 学習用のDataFrameを作成する
df_study = pl.concat(items = [df_numeric, df_dummy], how = "horizontal")
display(df_study.head())

# モデルを学習する
lr = linear_model.LinearRegression()
model = lr.fit(X = df_study, y = tips["tip"])

# 係数と切片を表示する
display(model.coef_)
display(model.intercept_)